# Uniswap Pair Candidates

In [1]:
import dijkstra
import sqlite3
import pandas as pd
import numpy as np

## Variables
pairs_db = r"raw_pairs.db"
usdt_address = r"0x382bB369d343125BfB2117af9c149795C6C65C50".lower()
eth_address = r"0x8F8526dbfd6E38E3D8307702cA8469Bae6C56C15".lower()

## Data import

Data is downloaded via uniswap contracts with a node script, and persisted locally into sqlite.

### Pairs

In [2]:
con = sqlite3.connect(pairs_db)

sql_query = """
  select
	p.[index],
	
	lower(json_extract(p.contentJson, '$.address')) as address,
	
	json_extract(p.contentJson, '$.reserves0') as reserves_0,
	lower(json_extract(p.contentJson, '$.token0.address')) as token_0,
	json_extract(p.contentJson, '$.token0.name') as name_0,
	json_extract(p.contentJson, '$.token0.symbol') as symbol_0,
	json_extract(p.contentJson, '$.token0.decimals') as decimals_0,
	
	json_extract(p.contentJson, '$.reserves1') as reserves_1,
	lower(json_extract(p.contentJson, '$.token1.address')) as token_1,
	json_extract(p.contentJson, '$.token1.name') as name_1,
	json_extract(p.contentJson, '$.token1.symbol') as symbol_1,
	json_extract(p.contentJson, '$.token1.decimals') as decimals_1
from Pairs p
where p.status = 2"""

df_pairs = pd.read_sql(sql_query, con)
dirty_length = len(df_pairs)
df_pairs = df_pairs[df_pairs['address'].notnull()]
clean_length = len(df_pairs)

df_pairs['reserves_0'] = df_pairs.apply(lambda row: int(row['reserves_0']), axis=1)
df_pairs['reserves_1'] = df_pairs.apply(lambda row: int(row['reserves_1']), axis=1)

print(f"Number of pairs: {clean_length}")
print(f"Number of pairs filtered: {dirty_length - clean_length}")
df_pairs.head()

Number of pairs: 3868
Number of pairs filtered: 15


,index,address,reserves_0,token_0,name_0,symbol_0,decimals_0,reserves_1,token_1,name_1,symbol_1,decimals_1
0,0,0x532ecec1d5525bf5fa40e6b24081a022fdb45824,2098143411681401169013,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,121470393205177919087,0xdf54b6c6195ea4d948d03bfd818d365cf175cfc2,OKB,OKB,18.0
1,1,0xa75bd9f086bbc1168b01fd5e750986b5170c2b26,42782629472030691834724,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,2392803072157813600988,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0
2,2,0xeb02a695126b998e625394e43dfd26ca4a75ce2b,1068506460359498346112,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,652661829219583438,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0
3,3,0x838a7a7f3e16117763c109d98c79ddcd69f6fd6e,1832835382353307814223,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,86060765024044528,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0
4,4,0x8009edebbbdeb4a3bb3003c79877fcd98ec7fb45,46518803724869702247889,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,432007300822685978161214,0x5fac926bf1e638944bb16fb5b787b5ba4bc85b0a,Jswap Finance Token,JF,18.0


### Number of pairs

In [3]:
len(df_pairs)

3868

### Tokens

## All tokens, indexed by address

In [4]:
df_tokens_0 = df_pairs[['token_0','name_0','symbol_0','decimals_0']].rename(columns={'token_0':'address','name_0':'name', 'symbol_0':'symbol','decimals_0':'decimals'}).set_index('address')
df_tokens_1 = df_pairs[['token_1','name_1','symbol_1','decimals_1']].rename(columns={'token_1':'address','name_1':'name', 'symbol_1':'symbol','decimals_1':'decimals'}).set_index('address')
df_tokens = pd.concat([df_tokens_0, df_tokens_1]).drop_duplicates().sort_index()
df_tokens['address'] = df_tokens.index

print("Number of tokens: " + str(len(df_tokens)))
df_tokens.head()


Number of tokens: 2278


,name,symbol,decimals,address
address,,,,
0x002375b85432723092f59725610a8936c05f6461,Dogger,Dogger,18.0,0x002375b85432723092f59725610a8936c05f6461
0x0047d146845009bca983727b0cf9558a0be8436b,OKlaeeb,OKlaeeb,18.0,0x0047d146845009bca983727b0cf9558a0be8436b
0x00505505a7576d6734704fabb16f41924e3e384b,TRXK,TRXK,18.0,0x00505505a7576d6734704fabb16f41924e3e384b
0x00665feb7f797b93dbf96f3f0c9df600926f12d6,OverSafemoon,OverSafemoon,2.0,0x00665feb7f797b93dbf96f3f0c9df600926f12d6
0x00bf4e801a230b8c48212440077c845ca51e626a,DoggerCoin,DoggerCoin,18.0,0x00bf4e801a230b8c48212440077c845ca51e626a


### Validation

Our simple valuation method requires that;

- There is a USDT token
- There is an ETH token
- There is a pair between USDT and ETH

NOTE: There are lots of scam tokens with the symbol/name of reputable tokens. We need to validate by ERC-20 contract address.


In [5]:
usdt = df_tokens[df_tokens['address'] == usdt_address].iloc[0]
usdt

name                                              USDT
symbol                                            USDT
decimals                                          18.0
address     0x382bb369d343125bfb2117af9c149795c6c65c50
Name: 0x382bb369d343125bfb2117af9c149795c6c65c50, dtype: object

In [6]:
eth = df_tokens[df_tokens['address'] == eth_address].iloc[0]
eth

name                                       Wrapped OKT
symbol                                            WOKT
decimals                                          18.0
address     0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15
Name: 0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15, dtype: object

In [7]:
usdt_eth_pair = df_pairs[((df_pairs['token_0'] == usdt_address) | (df_pairs['token_1'] == usdt_address)) & ((df_pairs['token_0'] == eth_address) | (df_pairs['token_1'] == eth_address))].iloc[0]
usdt_eth_pair

index                                                  1
address       0xa75bd9f086bbc1168b01fd5e750986b5170c2b26
reserves_0                       42782629472030691834724
token_0       0x382bb369d343125bfb2117af9c149795c6c65c50
name_0                                              USDT
symbol_0                                            USDT
decimals_0                                          18.0
reserves_1                        2392803072157813600988
token_1       0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15
name_1                                       Wrapped OKT
symbol_1                                            WOKT
decimals_1                                          18.0
Name: 1, dtype: object

## Valuation

To establish if a pair meets our baseline liquidity requirements for inclusion in arbitrage bot, we need to get the value of all tokens in USDT. 

If we build a graph of all the pairs, we can do (kind of) a flood-fill valuation from USDT => ETH => _everything else_

Graph structure:

- Token (addresses) are nodes
- Pairs are edges


In [8]:
graph = dijkstra.Graph()
for index, row in df_pairs.iterrows():
  # Edges are bi-directional, as you can buy/sell either asset in the pool
  graph.add_edge(row['token_0'], row['token_1'], 1)
  graph.add_edge(row['token_1'], row['token_0'], 1)

### Minimum distance from USDT to each token

Kind of interesting. Tokens with a distance > 2 will be omitted in our valuations (as they don't have a pair with USDT or ETH).

In [9]:
dijk = dijkstra.DijkstraSPF(graph, usdt_address)

df_tokens_routed = df_tokens.copy()
df_tokens_routed['distance'] = df_tokens_routed.apply(lambda row: dijk.get_distance(row['address']), axis=1)

df_tokens_routed.head()


,name,symbol,decimals,address,distance
address,,,,,
0x002375b85432723092f59725610a8936c05f6461,Dogger,Dogger,18.0,0x002375b85432723092f59725610a8936c05f6461,2.0
0x0047d146845009bca983727b0cf9558a0be8436b,OKlaeeb,OKlaeeb,18.0,0x0047d146845009bca983727b0cf9558a0be8436b,2.0
0x00505505a7576d6734704fabb16f41924e3e384b,TRXK,TRXK,18.0,0x00505505a7576d6734704fabb16f41924e3e384b,1.0
0x00665feb7f797b93dbf96f3f0c9df600926f12d6,OverSafemoon,OverSafemoon,2.0,0x00665feb7f797b93dbf96f3f0c9df600926f12d6,2.0
0x00bf4e801a230b8c48212440077c845ca51e626a,DoggerCoin,DoggerCoin,18.0,0x00bf4e801a230b8c48212440077c845ca51e626a,2.0


In [10]:
df_tokens_routed.groupby(['distance'])['distance'].count()

distance
0.0       1
1.0     383
2.0    1864
3.0      19
4.0       2
inf       9
Name: distance, dtype: int64

### Valuation 1/2: Calculating USD price of token that have a pair with USDT

In [11]:


def calculate_price_from_usdt(usdt_reserves, target_reserves, target_decimals):
  if (usdt_reserves == 0):
    return np.nan
  if (target_reserves == 0):
    return np.nan

  usdt_decimals = usdt['decimals']
  if (usdt_decimals > target_decimals):
    usdt_multiplier = 1
    target_multiplier = 10 ** (usdt_decimals - target_decimals)
  else:
    usdt_multiplier = 10 ** (target_decimals - usdt_decimals)
    target_multiplier = 1
  
  return (usdt_reserves * usdt_multiplier) / (target_reserves * target_multiplier)

df_priced_pairs_usdt_as_0 = df_pairs[df_pairs['token_0'] == usdt_address].copy()
df_priced_pairs_usdt_as_0['price_0'] = 1
df_priced_pairs_usdt_as_0['price_1'] = df_priced_pairs_usdt_as_0.apply(lambda row: calculate_price_from_usdt(row['reserves_0'], row['reserves_1'], row['decimals_1']), axis=1)

df_priced_pairs_usdt_as_1 = df_pairs[df_pairs['token_1'] == usdt_address].copy()
df_priced_pairs_usdt_as_1['price_0'] = df_priced_pairs_usdt_as_1.apply(lambda row: calculate_price_from_usdt(row['reserves_1'], row['reserves_0'], row['decimals_0']), axis=1)
df_priced_pairs_usdt_as_1['price_1'] = 1

df_priced_pairs_usdt = pd.concat([df_priced_pairs_usdt_as_0, df_priced_pairs_usdt_as_1])
df_priced_pairs_usdt.head()

,index,address,reserves_0,token_0,name_0,symbol_0,decimals_0,reserves_1,token_1,name_1,symbol_1,decimals_1,price_0,price_1
0,0,0x532ecec1d5525bf5fa40e6b24081a022fdb45824,2098143411681401169013,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,121470393205177919087,0xdf54b6c6195ea4d948d03bfd818d365cf175cfc2,OKB,OKB,18.0,1.0,17.272879
1,1,0xa75bd9f086bbc1168b01fd5e750986b5170c2b26,42782629472030691834724,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,2392803072157813600988,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,1.0,17.879712
2,2,0xeb02a695126b998e625394e43dfd26ca4a75ce2b,1068506460359498346112,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,652661829219583438,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0,1.0,1637.151757
3,3,0x838a7a7f3e16117763c109d98c79ddcd69f6fd6e,1832835382353307814223,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,86060765024044528,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,1.0,21296.991513
4,4,0x8009edebbbdeb4a3bb3003c79877fcd98ec7fb45,46518803724869702247889,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,432007300822685978161214,0x5fac926bf1e638944bb16fb5b787b5ba4bc85b0a,Jswap Finance Token,JF,18.0,1.0,0.107681


### Valuation 2/2: Calculating USD price of token that have a pair with ETH

In [12]:
usdt_eth_price_pair = df_priced_pairs_usdt[(df_priced_pairs_usdt['token_0'] == usdt_address) & (df_priced_pairs_usdt['token_1'] == eth_address)]
eth_usdt_price_pair = df_priced_pairs_usdt[(df_priced_pairs_usdt['token_0'] == eth_address) & (df_priced_pairs_usdt['token_1'] == usdt_address)]
eth_price_usdt = usdt_eth_price_pair.iloc[0]['price_1'] if len(usdt_eth_price_pair) else eth_usdt_price_pair.iloc[0]['price_0']
eth_price_usdt

def calculate_price_from_eth(eth_reserves, target_reserves, target_decimals):
  if (eth_reserves == 0):
    return np.nan
  if (target_reserves == 0):
    return np.nan

  eth_decimals = eth['decimals']
  if (eth_decimals > target_decimals):
    eth_multiplier = 1
    target_multiplier = 10 ** (eth_decimals - target_decimals)
  else:
    eth_multiplier = 10 ** (target_decimals - eth_decimals)
    target_multiplier = 1

  eth_reserves_as_usdt = eth_reserves * eth_price_usdt
  return (eth_reserves_as_usdt * eth_multiplier) / (target_reserves * target_multiplier)

df_priced_pairs_eth_as_0 = df_pairs[df_pairs['token_0'] == eth_address].copy()
df_priced_pairs_eth_as_0['price_0'] = eth_price_usdt
df_priced_pairs_eth_as_0['price_1'] = df_priced_pairs_eth_as_0.apply(lambda row: calculate_price_from_eth(row['reserves_0'], row['reserves_1'], row['decimals_1']), axis=1)

df_priced_pairs_eth_as_1 = df_pairs[df_pairs['token_1'] == eth_address].copy()
df_priced_pairs_eth_as_1['price_0'] = df_priced_pairs_eth_as_1.apply(lambda row: calculate_price_from_eth(row['reserves_1'], row['reserves_0'], row['decimals_0']), axis=1)
df_priced_pairs_eth_as_1['price_1'] = eth_price_usdt

df_priced_pairs_eth = pd.concat([df_priced_pairs_eth_as_0, df_priced_pairs_eth_as_1])
df_priced_pairs_eth.head()

,index,address,reserves_0,token_0,name_0,symbol_0,decimals_0,reserves_1,token_1,name_1,symbol_1,decimals_1,price_0,price_1
6,6,0x57075b512eead501982f353999b5a8a501ec8158,20,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,218984,0xd7bb0621335b854cde14477ed91e6825c3c325dd,BabyDoge,BabyDoge,9.0,17.879712,1.632970e-12
8,8,0x51d3c41988e72f0cd677aa9b1f72cd719129f6a7,0,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,0,0xf741d25e4fe06a673d2c867f303aed11bb95050c,minidoge,minidoge,9.0,17.879712,NaN
10,10,0x5c9467b14b40e5b10790b5dc1271e01fc91f352e,1,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,3598385,0xc5a60e0c8458b03f13fbce99db5e7d45c715cfb7,BabyShib,BabyShib,9.0,17.879712,4.968816e-15
13,13,0x82d1a2cc2258e0c2fee58d85f98289f4f193b138,1,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,1496045,0xfcbb7ceecd56149ed32e0eaec49ab0e73f5c5ee5,KODoge,KODoge,9.0,17.879712,1.195132e-14
16,16,0xc1b4e9ebd47ae994ec3e3f76dffd005214d13d07,9,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,114143,0xac578175f0dd5cf3d090645f9bc60d33cbd1c549,KODoge,KODoge,9.0,17.879712,1.409788e-12


In [13]:
df_priced_pairs = pd.concat([df_priced_pairs_usdt, df_priced_pairs_eth])
dirty_length = len(df_priced_pairs)

df_priced_pairs = df_priced_pairs[(np.isnan(df_priced_pairs['price_0']) == False) & (np.isnan(df_priced_pairs['price_1']) == False)]
clean_length = len(df_priced_pairs)

df_priced_pairs = df_priced_pairs.sort_index()

print(f"Number of pairs: {clean_length}")
print(f"Number of pairs filtered: {dirty_length - clean_length}")
df_priced_pairs.head()

Number of pairs: 2746
Number of pairs filtered: 702


,index,address,reserves_0,token_0,name_0,symbol_0,decimals_0,reserves_1,token_1,name_1,symbol_1,decimals_1,price_0,price_1
0,0,0x532ecec1d5525bf5fa40e6b24081a022fdb45824,2098143411681401169013,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,121470393205177919087,0xdf54b6c6195ea4d948d03bfd818d365cf175cfc2,OKB,OKB,18.0,1.0,17.272879
1,1,0xa75bd9f086bbc1168b01fd5e750986b5170c2b26,42782629472030691834724,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,2392803072157813600988,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,1.0,17.879712
1,1,0xa75bd9f086bbc1168b01fd5e750986b5170c2b26,42782629472030691834724,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,2392803072157813600988,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,1.0,17.879712
2,2,0xeb02a695126b998e625394e43dfd26ca4a75ce2b,1068506460359498346112,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,652661829219583438,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0,1.0,1637.151757
3,3,0x838a7a7f3e16117763c109d98c79ddcd69f6fd6e,1832835382353307814223,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,86060765024044528,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,1.0,21296.991513


## Liquidity

Now that we have priced everything in USDT, we can calculate liquidity in USDT - then we can analyze liquidity across pools!

In [14]:
df_pairs_final = df_priced_pairs.copy()
df_pairs_final['reserves_0_usd'] = df_pairs_final.apply(lambda row: row['reserves_0'] * row['price_0'], axis=1)
df_pairs_final['reserves_0_usd_$'] = df_pairs_final['reserves_0_usd'] / (10 ** usdt['decimals'])
df_pairs_final['reserves_1_usd'] = df_pairs_final.apply(lambda row: row['reserves_1'] * row['price_1'], axis=1)
df_pairs_final['reserves_1_usd_$'] = df_pairs_final['reserves_1_usd'] / (10 ** usdt['decimals'])
df_pairs_final['reserves_usd'] = df_pairs_final.apply(lambda row: row['reserves_0_usd'] + row['reserves_1_usd'], axis=1)
df_pairs_final['reserves_usd_$'] = df_pairs_final['reserves_usd'] / (10 ** usdt['decimals'])
df_pairs_final.head()

,index,address,reserves_0,token_0,name_0,symbol_0,decimals_0,reserves_1,token_1,name_1,symbol_1,decimals_1,price_0,price_1,reserves_0_usd,reserves_0_usd_$,reserves_1_usd,reserves_1_usd_$,reserves_usd,reserves_usd_$
0,0,0x532ecec1d5525bf5fa40e6b24081a022fdb45824,2098143411681401169013,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,121470393205177919087,0xdf54b6c6195ea4d948d03bfd818d365cf175cfc2,OKB,OKB,18.0,1.0,17.272879,2.098143e+21,2098.143412,2.098143e+21,2098.143412,4.196287e+21,4196.286823
1,1,0xa75bd9f086bbc1168b01fd5e750986b5170c2b26,42782629472030691834724,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,2392803072157813600988,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,1.0,17.879712,4.278263e+22,42782.629472,4.278263e+22,42782.629472,8.556526e+22,85565.258944
1,1,0xa75bd9f086bbc1168b01fd5e750986b5170c2b26,42782629472030691834724,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,2392803072157813600988,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,1.0,17.879712,4.278263e+22,42782.629472,4.278263e+22,42782.629472,8.556526e+22,85565.258944
2,2,0xeb02a695126b998e625394e43dfd26ca4a75ce2b,1068506460359498346112,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,652661829219583438,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0,1.0,1637.151757,1.068506e+21,1068.506460,1.068506e+21,1068.506460,2.137013e+21,2137.012921
3,3,0x838a7a7f3e16117763c109d98c79ddcd69f6fd6e,1832835382353307814223,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,86060765024044528,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,1.0,21296.991513,1.832835e+21,1832.835382,1.832835e+21,1832.835382,3.665671e+21,3665.670765


e.g. How many pools have > $250 TVL?

In [15]:
df_pairs_250 = df_pairs_final[df_pairs_final['reserves_usd_$'] > 250]
df_pairs_250

,index,address,reserves_0,token_0,name_0,symbol_0,decimals_0,reserves_1,token_1,name_1,symbol_1,decimals_1,price_0,price_1,reserves_0_usd,reserves_0_usd_$,reserves_1_usd,reserves_1_usd_$,reserves_usd,reserves_usd_$
0,0,0x532ecec1d5525bf5fa40e6b24081a022fdb45824,2098143411681401169013,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,121470393205177919087,0xdf54b6c6195ea4d948d03bfd818d365cf175cfc2,OKB,OKB,18.0,1.000000,1.727288e+01,2.098143e+21,2098.143412,2.098143e+21,2098.143412,4.196287e+21,4196.286823
1,1,0xa75bd9f086bbc1168b01fd5e750986b5170c2b26,42782629472030691834724,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,2392803072157813600988,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,1.000000,1.787971e+01,4.278263e+22,42782.629472,4.278263e+22,42782.629472,8.556526e+22,85565.258944
1,1,0xa75bd9f086bbc1168b01fd5e750986b5170c2b26,42782629472030691834724,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,2392803072157813600988,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,1.000000,1.787971e+01,4.278263e+22,42782.629472,4.278263e+22,42782.629472,8.556526e+22,85565.258944
2,2,0xeb02a695126b998e625394e43dfd26ca4a75ce2b,1068506460359498346112,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,652661829219583438,0xef71ca2ee68f45b9ad6f72fbdb33d707b872315c,ETHK,ETHK,18.0,1.000000,1.637152e+03,1.068506e+21,1068.506460,1.068506e+21,1068.506460,2.137013e+21,2137.012921
3,3,0x838a7a7f3e16117763c109d98c79ddcd69f6fd6e,1832835382353307814223,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,86060765024044528,0x54e4622dc504176b3bb432dccaf504569699a7ff,BTCK,BTCK,18.0,1.000000,2.129699e+04,1.832835e+21,1832.835382,1.832835e+21,1832.835382,3.665671e+21,3665.670765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3271,3271,0xc1310026a4ff02e788072e3fa7a24113e266dba0,1323244620188923481765,0x382bb369d343125bfb2117af9c149795c6c65c50,USDT,USDT,18.0,7017396201700101356443,0x66f4172e9e2341e8c4e90488b833c095b1057a9b,DYPI Token,DYPI,18.0,1.000000,1.885663e-01,1.323245e+21,1323.244620,1.323245e+21,1323.244620,2.646489e+21,2646.489240
3443,3443,0xf9c658e11d08202e26d98fb2df7f6052566d76f3,107234301024509694853,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,471488651595398053,0xf94f60f6caa260d83b805a0a2b399f64c1e84443,oktzilla,oktzilla,9.0,17.879712,4.066521e-06,1.917318e+21,1917.318404,1.917318e+12,0.000002,1.917318e+21,1917.318406
3486,3486,0xf07b61f1e076f5879a0f60158b375b857488c179,405713243767336938797,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,298465759669041726882238113860386,0xaf655692e57b6bf3840b86f7eeebb4eecbc62d63,OKXMANCITY,OKXMANCITY,18.0,17.879712,2.430442e-11,7.254036e+21,7254.035897,7.254036e+21,7254.035897,1.450807e+22,14508.071794
3571,3571,0xd816d7f5db677f9fe23aad0810283858479b18a6,35760690822872170505,0x8f8526dbfd6e38e3d8307702ca8469bae6c56c15,Wrapped OKT,WOKT,18.0,124222924618919040544,0xcd4ce655337a9ef9ea26ce50abb910aeddebb6db,Okfly Plus New,Okfly New,18.0,17.879712,5.147124e+00,6.393908e+20,639.390848,6.393908e+20,639.390848,1.278782e+21,1278.781696
